In [0]:
--------WITH CLAUSE, CET, Sub-Query Factoring

In [0]:
%sql

create table mydb.emp
( emp_ID int
, emp_NAME varchar(50)
, SALARY int);

In [0]:
%sql

insert into mydb.emp values(101, 'Mohan', 40000);
insert into mydb.emp values(102, 'James', 50000);
insert into mydb.emp values(103, 'Robin', 60000);
insert into mydb.emp values(104, 'Carol', 70000);
insert into mydb.emp values(105, 'Alice', 80000);
insert into mydb.emp values(106, 'Jimmy', 90000);

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

create table mydb.sales
(
	store_id  		int,
	store_name  	varchar(50),
	product			varchar(50),
	quantity		int,
	cost			int
);

In [0]:
%sql
insert into mydb.sales values
(1, 'Apple Originals 1','iPhone 12 Pro', 1, 1000),
(1, 'Apple Originals 1','MacBook pro 13', 3, 2000),
(1, 'Apple Originals 1','AirPods Pro', 2, 280),
(2, 'Apple Originals 2','iPhone 12 Pro', 2, 1000),
(3, 'Apple Originals 3','iPhone 12 Pro', 1, 1000),
(3, 'Apple Originals 3','MacBook pro 13', 1, 2000),
(3, 'Apple Originals 3','MacBook Air', 4, 1100),
(3, 'Apple Originals 3','iPhone 12', 2, 1000),
(3, 'Apple Originals 3','AirPods Pro', 3, 280),
(4, 'Apple Originals 4','iPhone 12 Pro', 2, 1000),
(4, 'Apple Originals 4','MacBook pro 13', 1, 2500);


num_affected_rows,num_inserted_rows
11,11


In [0]:
%sql

---fecth employee who earn more than avearage salary of all employess
---- Sql query can also start with with clause

with avg_salary(avg_sal) as
        (select avg(salary) from mydb.emp)
select * 
from mydb.emp e,avg_salary av
where e.salary > av.avg_sal

emp_ID,emp_NAME,SALARY,avg_sal
104,Carol,70000,65000.0
105,Alice,80000,65000.0
106,Jimmy,90000,65000.0


In [0]:
%sql
----2. find the average sales w,r,t all the stores -- Avg_sales
select avg(total_sales_per_stores) as  avg_sales_for_all_stores
 from (  select s.store_id,sum(cost) as total_sales_per_stores
         from mydb.sales s
          group by s.store_id) x

avg_sales_for_all_stores
3290.0


In [0]:
%sql

----3. Find the stores where total_sales>avg_sales of all stores.

select *
  from (  select s.store_id,sum(cost) as total_sales_per_stores
          from mydb.sales s
          group by s.store_id) total_sales
          
    join (   select avg(total_sales_per_stores) as  avg_sales_for_all_stores
             from (  select s.store_id,sum(cost) as total_sales_per_stores
             from mydb.sales s
              group by s.store_id) x) avg_sales
              
        on total_sales.total_sales_per_stores > avg_sales.avg_sales_for_all_stores

store_id,total_sales_per_stores,avg_sales_for_all_stores
4,3500,3290.0
3,5380,3290.0


In [0]:
%sql
---find store whose sales where better than the average sales across all stores.
--- 1. Total sales per each stores ---total_sales

select s.store_id,sum(cost) as total_sales_per_stores
from mydb.sales s
group by s.store_id
----2. find the average sales w,r,t all the stores -- Avg_sales


----3. Find the stores where total_sales>avg_sales of all stores.

--with 

--select * from mydb.sales;

store_id,total_sales_per_stores
4,3500
3,5380
1,3280
2,1000


In [0]:
%sql
---- same query with WITH Clause i.e. CET

with total_sales (store_id,total_sales_per_stores)  as
        (  select s.store_id,sum(cost) as total_sales_per_stores
           from mydb.sales s
           group by s.store_id),
       avg_sales(avg_sales_for_all_stores)  as
           ( select avg(total_sales_per_stores) as  avg_sales_for_all_stores
             from total_sales)
             
select *
from total_sales ts
join avg_sales av
on ts.total_sales_per_stores > av.avg_sales_for_all_stores
           

    

store_id,total_sales_per_stores,avg_sales_for_all_stores
4,3500,3290.0
3,5380,3290.0


In [0]:
%sql


-- QUERY 1 :
drop table emp;
create table emp
( emp_ID int
, emp_NAME varchar(50)
, SALARY int);

insert into emp values(101, 'Mohan', 40000);
insert into emp values(102, 'James', 50000);
insert into emp values(103, 'Robin', 60000);
insert into emp values(104, 'Carol', 70000);
insert into emp values(105, 'Alice', 80000);
insert into emp values(106, 'Jimmy', 90000);

select * from emp;

with avg_sal(avg_salary) as
		(select cast(avg(salary) as int) from emp)
select *
from emp e
join avg_sal av on e.salary > av.avg_salary





-- QUERY 2 :
DrOP table sales ;
create table sales
(
	store_id  		int,
	store_name  	varchar(50),
	product			varchar(50),
	quantity		int,
	cost			int
);
insert into sales values
(1, 'Apple Originals 1','iPhone 12 Pro', 1, 1000),
(1, 'Apple Originals 1','MacBook pro 13', 3, 2000),
(1, 'Apple Originals 1','AirPods Pro', 2, 280),
(2, 'Apple Originals 2','iPhone 12 Pro', 2, 1000),
(3, 'Apple Originals 3','iPhone 12 Pro', 1, 1000),
(3, 'Apple Originals 3','MacBook pro 13', 1, 2000),
(3, 'Apple Originals 3','MacBook Air', 4, 1100),
(3, 'Apple Originals 3','iPhone 12', 2, 1000),
(3, 'Apple Originals 3','AirPods Pro', 3, 280),
(4, 'Apple Originals 4','iPhone 12 Pro', 2, 1000),
(4, 'Apple Originals 4','MacBook pro 13', 1, 2500);

select * from sales;


-- Find total sales per each store
select s.store_id, sum(s.cost) as total_sales_per_store
from sales s
group by s.store_id;


-- Find average sales with respect to all stores
select cast(avg(total_sales_per_store) as int) avg_sale_for_all_store
from (select s.store_id, sum(s.cost) as total_sales_per_store
	from sales s
	group by s.store_id) x;



-- Find stores who's sales where better than the average sales accross all stores
select *
from   (select s.store_id, sum(s.cost) as total_sales_per_store
				from sales s
				group by s.store_id
	   ) total_sales
join   (select cast(avg(total_sales_per_store) as int) avg_sale_for_all_store
				from (select s.store_id, sum(s.cost) as total_sales_per_store
		  	  		from sales s
			  			group by s.store_id) x
	   ) avg_sales
on total_sales.total_sales_per_store > avg_sales.avg_sale_for_all_store;



-- Using WITH clause
WITH total_sales as
		(select s.store_id, sum(s.cost) as total_sales_per_store
		from sales s
		group by s.store_id),
	avg_sales as
		(select cast(avg(total_sales_per_store) as int) avg_sale_for_all_store
		from total_sales)
select *
from   total_sales
join   avg_sales
on total_sales.total_sales_per_store > avg_sales.avg_sale_for_all_store;
